In [1]:
import pandas as pd
import requests, json
import numpy as np
import folium
import re
import json
import datetime
import xml.etree.ElementTree as Xml

In [2]:
minlon, minlat, maxlon, maxlat = 77.7519486464,12.9833797192,77.7528560462,12.9842016832

minlat=str(minlat)
minlon=str(minlon)
maxlat=str(maxlat)
maxlon=str(maxlon)

In [3]:
hermap_filename = 'data_comparision/heremap.json'
sumo_filename = 'data_comparision/original_sumo.net.xml'

In [4]:
with open(hermap_filename) as f:
    heremap_json = json.load(f)

In [5]:
e = Xml.parse(sumo_filename).getroot()
for i in e:
    print(i)

<Element 'location' at 0x7fd3ad43fae8>
<Element 'type' at 0x7fd3ad43f728>
<Element 'type' at 0x7fd3ad43f9f8>
<Element 'type' at 0x7fd3ad43fb88>
<Element 'type' at 0x7fd3ad43fb38>
<Element 'type' at 0x7fd3ad43fd18>
<Element 'type' at 0x7fd3ad43fe08>
<Element 'type' at 0x7fd3ad43fe58>
<Element 'type' at 0x7fd3ad43fea8>
<Element 'type' at 0x7fd3ad43fef8>
<Element 'type' at 0x7fd3ad43ff48>
<Element 'type' at 0x7fd3ad43ff98>
<Element 'type' at 0x7fd3ad3d2048>
<Element 'type' at 0x7fd3ad3d2098>
<Element 'type' at 0x7fd3ad3d20e8>
<Element 'type' at 0x7fd3ad3d2138>
<Element 'type' at 0x7fd3ad3d2188>
<Element 'type' at 0x7fd3ad3d2228>
<Element 'type' at 0x7fd3ad3d2278>
<Element 'type' at 0x7fd3ad3d22c8>
<Element 'type' at 0x7fd3ad3d2318>
<Element 'type' at 0x7fd3ad3d2368>
<Element 'type' at 0x7fd3ad3d23b8>
<Element 'type' at 0x7fd3ad3d2408>
<Element 'type' at 0x7fd3ad3d2458>
<Element 'type' at 0x7fd3ad3d24a8>
<Element 'type' at 0x7fd3ad3d24f8>
<Element 'type' at 0x7fd3ad3d2548>
<Element 'type' 

In [7]:
import traffic_from_api
dd = traffic_from_api.HereMapInfo(minlat,minlon, maxlat,  maxlon)
a = dd.build_traffic_flow('dd')

In [8]:
a

{'CREATED_TIMESTAMP': '2018-06-25T10:17:48.000+0000',
 'MAP_VERSION': '',
 'RWS': [{'EBU_COUNTRY_CODE': '5',
   'EXTENDED_COUNTRY_CODE': 'F2',
   'MAP_VERSION': '201802',
   'RW': [{'DE': 'NH-207',
     'FIS': [{'FI': [{'CF': [{'CN': 0.71,
           'FF': 36.0,
           'JF': 4.02296,
           'SP': 23.12,
           'SU': 23.12,
           'TY': 'TR'}],
         'SHP': [{'FC': 3, 'value': ['12.96984,77.74992 12.97022,77.74997 ']},
          {'FC': 3, 'value': ['12.97022,77.74997 12.9703,77.74998 ']},
          {'FC': 3, 'value': ['12.9703,77.74998 12.9704,77.75 ']},
          {'FC': 3, 'value': ['12.9704,77.75 12.97075,77.75006 ']},
          {'FC': 3, 'value': ['12.97075,77.75006 12.97083,77.75007 ']},
          {'FC': 3, 'value': ['12.97083,77.75007 12.97092,77.75008 ']},
          {'FC': 3, 'value': ['12.97092,77.75008 12.97102,77.75009 ']},
          {'FC': 3, 'value': ['12.97102,77.75009 12.97143,77.75014 ']},
          {'FC': 3, 'value': ['12.97143,77.75014 12.97202,77.7502

In [23]:
def create_data_1(dictionary):
        
    RWS=dictionary['RWS']
    l=[]
    for rw in RWS:
        EBU_COUNTRY_CODE=rw['EBU_COUNTRY_CODE']
        EXTENDED_COUNTRY_CODE=rw['EXTENDED_COUNTRY_CODE']
        UNITS=rw['UNITS']
        for elem in rw['RW']:
            DE=elem['DE']
            LI=elem['LI']
            PBT=elem['PBT']
            mid=elem['mid']
            FIS=elem['FIS']
            for FI in FIS:
                elem_1=FI['FI']
                for elem_2 in elem_1:
                    LE=elem_2['TMC']['LE']
                    PC=elem_2['TMC']['PC']
                    QD=elem_2['TMC']['QD']
                    SHP=elem_2['SHP']
                    CF=elem_2['CF']
                    for elem_3 in CF:
                        FF=elem_3['FF']
                        CN=elem_3['CN']                    
                        JF=elem_3['JF']
                        SP=elem_3['SP']
                        TY=elem_3['TY']
                        l.append([EBU_COUNTRY_CODE,EXTENDED_COUNTRY_CODE,UNITS,DE,LI,PBT,mid,LE,PC,QD,SHP,FF,CN,JF,SP,TY])
    clms=['EBU_COUNTRY_CODE','EXTENDED_COUNTRY_CODE','UNITS','DE','LI','PBT','mid','LE','PC','QD','SHP','FF','CN','JF','SP','TY']                    
    df=pd.DataFrame(l,columns=clms)
    df['PBT']=datetime.datetime.now()
    #print(json.dumps(rws_list, indent=2))
#     if path_to_save:
#         if os.path.isfile(path_to_save):
#             df.to_csv(path_to_save,mode='a+',header=False,index=False)
#         else:
#             df.to_csv(path_to_save,mode='w+',index=False)
#     else:
    df['SHP'] = df['SHP'].astype(str)
    return df  

def traffic_map_across_city_historical(traffic_data):
    
    traffic_hour= traffic_data
    df_jaipur=traffic_data
    df_ori=df_jaipur
    list_shp = df_jaipur['SHP'].unique()
    list_mean = []
    start_point=[]
    end_point=[]
    for s in list_shp:
        tmp = re.findall("[+-]?\d*\.\d+", s)
        #print(tmp)
        tmp=[float(i) for i in tmp]
        tmp2 = []
        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        a = np.array(tmp2)
        list_mean.append(list(np.mean(a, axis=0)))
        start_point.append(tmp2[0])
        end_point.append(tmp2[-1])
    df_t = pd.DataFrame()
    df_t['SHP'] = list_shp
    df_t['mean'] = list_mean
    df_final =df_jaipur.merge(df_t,on='SHP')
    df_final['latitude'] = np.array(df_final['mean'].values.tolist())[:,0]
    df_final['longitude'] = np.array(df_final['mean'].values.tolist())[:,1]
    hh=pd.pivot_table(df_final,index=['SHP','PC','DE'],values='JF',aggfunc='median').reset_index()
    da1=hh.sort_values('JF',ascending=False)
    #da2=da1[da1['JF']>5]
    da2=da1

    data_2=da2
    data_2=data_2
    import branca.colormap as cm

    colormap = cm.LinearColormap(['green','yellow','red'],vmin=0,vmax=10)
    colormap.caption="Jamfactor_levels"
#     lat_long=df[['latitude','longitude']].values[0].tolist()
    x=folium.Map(location=[12.9380131671,77.6319900345],zoom_start=10)
    # x

    for i in data_2.index.tolist():
        color_grad_val=data_2['JF'][i]
        tmp2=[]
        #tmp=re.findall("\d*\.\d+", data_2['SHP'][i])
        tmp = re.findall("[+-]?\d*\.\d+", data_2['SHP'][i])
        #print(tmp)
        tmp=[float(i) for i in tmp]

        it = iter(tmp)
        for z in it:
            tmp2.append([z, next(it)])
        y=folium.PolyLine(locations=tmp2,color=colormap.rgb_hex_str(color_grad_val),opacity=0.75,weight=7)
        x.add_children(y)
        
    
    df= df_final[['latitude','longitude','PC']].drop_duplicates()

    l = df[['latitude','longitude']].values.tolist()

    #map_osm = folium.Map(location=[38.946241233265095, -77.04115322627564],tiles='CartoDB Positron',
    #                     zoom_start=12)
    for i,k in zip(l,df['PC']):

        marker = folium.Marker(i,popup=str(k),icon=folium.Icon())

        x.add_child(marker)

    

    x
    display(x)

In [25]:
heremap_data = create_data_1(heremap_json)
traffic_map_across_city_historical(heremap_data)

/home/utkarsh/.virtualenvs/Traffic_Simulation/lib/python3.5/site-packages/ipykernel_launcher.py:97: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.
